In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image, ExifTags
import datetime as dt
import pandas as pd
from lxml import etree
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
def matlabdn2datetime(dn):
    """http://stackoverflow.com/questions/13965740/converting-matlabs-datenum-format-to-python"""
    day = dt.datetime.fromordinal(int(dn))
    dayfrac = dt.timedelta(days=dn%1) - dt.timedelta(days = 366)
    return day + dayfrac

def datetime2matlabdn(dtn):
   '''http://stackoverflow.com/questions/8776414/python-datetime-to-matlab-datenum'''
   mdn = dtn + dt.timedelta(days = 366)
   frac_seconds = (dtn-dt.datetime(dtn.year,dtn.month,dtn.day,0,0,0)).seconds / (24.0 * 60.0 * 60.0)
   frac_microseconds = dtn.microsecond / (24.0 * 60.0 * 60.0 * 1000000.0)
   return mdn.toordinal() + frac_seconds + frac_microseconds

dtn=dt.datetime(2012, 2, 13, 6, 56, 2, 619000)
dn=datetime2matlabdn(dtn)
dtn2 = matlabdn2datetime(dn)
dn2 =datetime2matlabdn(dtn2)
print(dtn2,dn,dn2)
print(dtn+dt.timedelta(minutes=5.))

2012-02-13 06:56:02.618997 734912.2889192014 734912.2889192014
2012-02-13 07:01:02.619000


In [4]:
# This is where the elements of the xml (gpx) file are defined
namespace = {'def': 'http://www.topografix.com/GPX/1/1'}

# get list of .gpx files the requested path (I have them in another folder at the same level)
path = '../uas'
glist=[os.path.join(path,g) for g in os.listdir(path) if ( g.endswith('.gpx') or g.endswith('.GPX'))]
print(glist)

['../uas\\solo2.gpx', '../uas\\solo3.gpx', '../uas\\solo4.gpx', '../uas\\solo5.gpx', '../uas\\solo6.gpx', '../uas\\solo7.gpx', '../uas\\solo9.gpx']


In [6]:
#parse the gpx file and report start and end time
#TODO - Do something more useful with the time zone info
for f in glist:
    tree = etree.parse(f)
    elist = tree.xpath('./def:trk//def:trkpt//def:time',namespaces=namespace)
    fmt = '%Y-%m-%dT%H:%M:%S-04:00' #2017-05-04T14:14:12-04:00
    time = [dt.datetime.strptime(e.text, fmt) for e in elist]
    print(f)
    print(time[0],time[-1], np.shape(time))

../uas\solo2.gpx
2017-05-04 14:14:00 2017-05-04 14:28:38 (14466,)
../uas\solo3.gpx
2017-05-04 13:52:31 2017-05-04 14:14:00 (9911,)
../uas\solo4.gpx
2017-05-04 13:29:14 2017-05-04 13:52:31 (7640,)
../uas\solo5.gpx
2017-05-04 13:01:35 2017-05-04 13:29:13 (10633,)
../uas\solo6.gpx
2017-05-04 12:41:01 2017-05-04 13:01:34 (8320,)
../uas\solo7.gpx
2017-05-04 12:23:21 2017-05-04 12:41:01 (7790,)
../uas\solo9.gpx
2017-05-04 12:03:40 2017-05-04 12:18:40 (8361,)


In [ ]:
# get a list of images files
path = '..\uas'
flist=[os.path.join(path,f) for f in os.listdir(path) if ( f.endswith('.jpg') or f.endswith('.JPG'))]

# read and print the time stamp
for f in flist:
    t=Image.open(f)._getexif()[36867]
    print(f,t)

# this does the same thing but makes a datetime array   
fmt = '%Y:%m:%d %H:%M:%S' #2017-05-04T14:14:12-04:00
pictime = [datetime.strptime(Image.open(f)._getexif()[36867], fmt) for f in flist]

In [ ]:
elist = tree.xpath('./def:trk//def:trkpt',namespaces=namespace)
lonlat = [e.values() for e in elist]
lonlat = np.array(lonlat,dtype="float")
print(lonlat[0])
print(np.shape(lonlat))

# This works
elist = tree.xpath('./def:trk//def:trkpt//def:time',namespaces=namespace)
fmt = '%Y-%m-%dT%H:%M:%S-04:00' #2017-05-04T14:14:12-04:00
time = [datetime.strptime(e.text, fmt) for e in elist]
print(time[0],time[-1], np.shape(time))


plt.plot(lonlat[:,1],lonlat[:,0])
plt.show()



# This works
elist = tree.xpath('./def:trk//def:trkpt//def:ele',namespaces=namespace)
ele = np.array([float(e.text) for e in elist])
print(ele[0], np.shape(ele))
print(np.shape(elist))

# This works
elist = tree.xpath('./def:trk//def:trkpt//def:ele2',namespaces=namespace)
ele2 = np.array([float(e.text) for e in elist])
print(ele2[0], np.shape(ele2))
print(np.shape(elist))

# get a list of images files
path = '.'
flist=[os.path.join(path,f) for f in os.listdir(path) if ( f.endswith('.jpg') or f.endswith('.JPG'))]

# print the time stamp
for f in flist:
    t=Image.open(f)._getexif()[36867]
    print(f,t)



# here is how to get all of the exif info:
# img = Image.open(flist[0])
# exif_data = img._getexif()
# exif = { ExifTags.TAGS[k]: v for k, v in img._getexif().items() if k in ExifTags.TAGS }
# print(exif)